# Step 8: Machine Learning/Deep Learning Prototype

## Learning Objective

- Find a machine learning or deep learning approach that works for the problem to be solved.
- Implement a prototype of the approach in a Jupyter notebook. 
- Demonstrate the viability of the approach.

### Prerequisites
- Python 3.9 
- Tensorflow - installed via Anaconda (https://www.anaconda.com/products/individual)
- Download yolov4.weights file: https://drive.google.com/open?id=1cewMfusmPjYWbrnuJRuKhPMwRe_b9PaT
and place in a "../yolo/weights/" folder
- opencv

Setup of folders and file paths used throughout this notebook. 
Note: this notebook calls a files/folders that are one folder above it. For it work, the entire git repo must be checked out as it uses submodules

In [1]:
from sys import path

os.chdir(os.path.dirname(path[0]))

If needed, set "get_requirements = True" to install necessary requirements used:

In [2]:
get_requirements = False
if (get_requirements == True):
    !pip install -r ../requirements.txt

Copy folders & files that will be used later to a tmp folder:

In [3]:
# using custom class for this project:
from utils.folder_utils import FolderUtils
from shutil import copy2

# copy folders that will be used by the algorithm
FolderUtils.CopyFolders('tensorflow_yolov4_tflite/data/', 'tmp/data') 
FolderUtils.CopyFolders('tensorflow_yolov4_tflite/core/', 'tmp/core') 
FolderUtils.CopyFolders('yolov4-deepsort/model_data/','tmp/model_data') 
FolderUtils.CopyFolders('utils','tmp/utils') 
FolderUtils.CopyFolders('deep_sort/deep_sort','tmp/deep_sort/deep_sort') 
FolderUtils.CopyFolders('deep_sort/tools','tmp/tools') 
FolderUtils.CopyFolders('yolo/weights','tmp/weights') 

copy2('yolo/darknet/data/coco.names', 'tmp/coco.names')
copy2('tensorflow_yolov4_tflite/save_model.py', 'tmp/save_model.py')
copy2('tensorflow_yolov4_tflite/save_model.py', 'tmp/save_model.py')
os.chdir('tmp')

Copied folder tensorflow_yolov4_tflite/data/ --> tmp/data
Copied folder tensorflow_yolov4_tflite/core/ --> tmp/core
Copied folder yolov4-deepsort/model_data/ --> tmp/model_data
Copied folder utils --> tmp/utils
Copied folder deep_sort/deep_sort --> tmp/deep_sort/deep_sort
Copied folder deep_sort/tools --> tmp/tools
Copied folder yolo/weights --> tmp/weights


Importing necessary libraries:

In [4]:

import os
from yaml import SafeLoader, load
import pandas as pd
import os
import cv2


# custom classes developed for this project:
from utils.deepsort_yolo import DeepsortYolo
from utils.video_utils import VideoUtils

Variables Setup: 

In [5]:
# setup
dataset_dir_path = '../../datasets/VIRAT/'
image_ext = '.jpg'
video_max_frames = 2000

#video
video_ext = '.mp4'
video_name = 'VIRAT_S_050000_07_001014_001126'
video_name_orig = video_name + video_ext
video_dest_path = './../processed/' +  video_name + '/'
video_src_path = dataset_dir_path + 'Videos/Ground/'

using_yml = True

# annotations
saved_csv = video_dest_path + 'df_bbox.csv'


yml_video_name = 'gt'
yolo_video_name = 'yolo'

annotations_path = dataset_dir_path + 'viratannotations/train/' + video_name +'/'
# annotations_path = dataset_dir_path + 'viratannotations/validate/' + video_name +'/'
ann_activities_file = annotations_path + video_name + '.activities.yml'
ann_geom_file = annotations_path + video_name + '.geom.yml'
ann_regions_file = annotations_path + video_name + '.regions.yml'
ann_types_file = annotations_path + video_name + '.types.yml'


src_video = os.path.join(video_src_path, video_name_orig)
gt_video_name = yml_video_name + '_' # + '.mp4'#video_ext
yolo_video_name = yolo_video_name + '_'# + '.mp4'#video_ext

Using https://github.com/zyerusha/tensorflow-yolov4-tflite to convert YOLO weights to tensorflow:


In [6]:
# If not done already, Convert weights to tensorflow model
if not os.path.exists("checkpoints/yolov4"):
    # current_dir  = os.getcwd()
    # os.chdir("tensorflow_yolov4_tflite/")
    !python save_model.py --model yolov4 --weights ./weights/yolov4.weights --output ./checkpoints/yolov4
    # os.chdir(current_dir)


In [7]:
# # Create directory to store new video
# if not os.path.exists(video_dest_path):
#     os.makedirs(video_dest_path)


The following code loads a dataframe with annotation data from either a yaml or from a csv file. The csv is created after the first yaml reading on the data. This provides  faster loading of data on a rerun of this notebook.

In [8]:
# using annotations:
print(f"Loading annotations for {video_name_orig}...")
def add_category_type(row):
  id = row['object_id']
  val = type_df.loc[type_df['id'] == id, 'category'].iloc[0]
  return val


if os.path.exists(saved_csv):
  df_bbox = pd.read_csv(saved_csv)
else:
    # Read categories from annotation file
    with open(ann_types_file) as yaml_file:
        yaml_contents = load(yaml_file, Loader=SafeLoader)
    yaml_df = pd.json_normalize(yaml_contents)
    yaml_df
    for col in yaml_df.columns:
        type_name = col.split('.')[-1]
        if not (type_name == 'id1'):
            yaml_df.loc[yaml_df[col] == 1, col] = type_name
    
    yaml_df = yaml_df[yaml_df['types.id1'].notna()].reset_index().dropna(axis=1, how='all')  
    type_df = yaml_df.ffill(axis=1).iloc[:,-1].to_frame(name='category')
    type_df.insert(0, "id", yaml_df['types.id1'])

    # Read bounding boxes from annotation file
    with open(ann_geom_file) as yaml_file:
        yaml_contents = load(yaml_file, Loader=SafeLoader)
    yaml_df = pd.json_normalize(yaml_contents)

    df_bbox = yaml_df[['geom.id1','geom.ts0','geom.ts1','geom.g0']].dropna().reset_index()
    df_bbox.rename(columns={'geom.id1': 'object_id', 'geom.ts0': 'frame_id','geom.ts1': 'time_sec', 'geom.g0': 'bbox'}, inplace=True)
    df_bbox['bbox'] = df_bbox['bbox'].str.split()
    df_tmp = pd.DataFrame(df_bbox['bbox'].to_list(), columns = ['bb_left', 'bb_top', 'bb_right', 'bb_bottom'])
    df_bbox = pd.concat([df_bbox, df_tmp], axis=1).drop(columns=['bbox'])

    df_bbox['category'] = df_bbox.apply(lambda row: add_category_type(row), axis=1) 
    df_bbox.drop(columns=['index'], axis=1, inplace=True)
    df_bbox.to_csv(saved_csv, index = False)
    

df_bbox.head(10)

Loading annotations for VIRAT_S_050000_07_001014_001126.mp4...


,object_id,frame_id,time_sec,bb_left,bb_top,bb_right,bb_bottom,category
0,0.0,0.0,0.000000,485,743,653,914,Vehicle
1,0.0,1.0,0.033333,489,748,657,919,Vehicle
2,0.0,2.0,0.066667,488,747,656,918,Vehicle
3,0.0,3.0,0.100000,488,747,656,918,Vehicle
4,0.0,4.0,0.133333,488,747,656,918,Vehicle
5,0.0,5.0,0.166667,488,746,656,917,Vehicle
6,0.0,6.0,0.200000,488,746,656,917,Vehicle
7,0.0,7.0,0.233333,488,746,656,917,Vehicle
8,0.0,8.0,0.266667,488,745,656,916,Vehicle
9,0.0,9.0,0.300000,488,745,656,916,Vehicle


Instantiating the classes that will used to process the video

In [9]:
vUtils = VideoUtils() 
deepsortYolo  = DeepsortYolo()

In [10]:


start_time = 0
video_duration=1#None
video_in = cv2.VideoCapture(src_video)
if video_in.isOpened():
    fps, total_frames, frame_size = VideoUtils.GetVideoData(video_in)
    start_count, end_count = VideoUtils.GetStartEndCount(fps, total_frames, start_time, video_duration)
    video_duration = int(end_count / fps)
    video_in.release()


Total frames in video: 3351 @ 30 frames/sec


In [11]:
yolo_filename = os.path.join(video_dest_path, VideoUtils.AddTimestampToName(yolo_video_name, start_time, video_duration))
gt_filename = os.path.join(video_dest_path, VideoUtils.AddTimestampToName(gt_video_name, start_time, video_duration))

tracker_file_csv = yolo_filename + '.csv'
tracker_file_mp4 = yolo_filename + '.mp4'
gt_file_mp4 = gt_filename + '.mp4'
if not os.path.exists(tracker_file_csv):
    model_filename = 'model_data/mars-small128.pb'
    yolo_weights_filename = './checkpoints/yolov4'
    tracker_video_out, trk_bbox = deepsortYolo.ProcessVideo(yolo_weights_filename, model_filename, src_video, video_dest_path, tracker_file_mp4, start_time_sec=start_time, duration_sec=video_duration, save_images=False)
    trk_bbox.to_csv(tracker_file_csv, index=False)

../../datasets/VIRAT/Videos/Ground/VIRAT_S_050000_07_001014_001126.mp4
opened video
Total frames in video: 3351 @ 30 frames/sec
Created frame id 25, 0.83 sec in video; Objects Cnt: 12 completed:  83.3 %
Done: Created video: ./../processed/VIRAT_S_050000_07_001014_001126/0-1_yolo_.mp4


Reload dataframe that was previously stored in a csv.

In [12]:
trk_bbox = pd.read_csv(tracker_file_csv)
trk_bbox.head(10)

,Frame,bb_left,bb_top,bb_right,bb_bottom,category,object_id
0,2,921.0,276.0,1028.0,364.0,car,1.0
1,2,797.0,121.0,908.0,195.0,car,2.0
2,2,1069.0,457.0,1180.0,545.0,car,3.0
3,2,1468.0,720.0,1496.0,803.0,person,4.0
4,2,1220.0,647.0,1421.0,830.0,truck,5.0
5,2,505.0,772.0,648.0,908.0,truck,6.0
6,2,979.0,348.0,1126.0,474.0,car,7.0
7,2,453.0,106.0,623.0,293.0,truck,8.0
8,2,873.0,198.0,993.0,290.0,car,9.0
9,2,1144.0,350.0,1170.0,410.0,person,10.0
